<a href="https://colab.research.google.com/github/sohrab-namazi/Genetic-Algorithms/blob/main/GA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Author: Sohrab Namazinia

# TSP by GA

In [3]:
from math import inf 

class graph:

  def __init__(self, node_count):
    self.node_count = node_count
    self.nodes = []
    for i in range(self.node_count):
      node = []
      for j in range(self.node_count):
        node.append(inf)
      self.nodes.append(node)

  def add_edge(self, a, b, w):
    self.nodes[a - 1][b - 1] = w
    self.nodes[b - 1][a - 1] = w
  
  def print_graph(self):
    print("Graph:\n")
    for i in range(self.node_count):
      print(str(i + 1) + ": " + str(self.nodes[i]))
    print("\n")

In [5]:
from math import inf
import random

# structure for each member of population
class individual:

  def __init__(self, pattern, fitness = 0):
    self.pattern = pattern
    self.fitness = fitness
  

class GA:

  def __init__(self, graph, population_count, parents_ratio, cross_over_parent1_ratio, recombination_low_fitness_ratio, cross_over_possibility, mutation_possibility, max_generation_count):
    self.graph = graph
    self.population = []
    self.population_count = population_count
    self.base_pattern = []
    self.recombination_low_fitness_count = int(self.population_count * recombination_low_fitness_ratio) 
    self.parents_count = int(self.population_count * parents_ratio)
    self.patterns_length = self.graph.node_count
    self.mutation_possibility = mutation_possibility
    self.max_generation_count = max_generation_count
    self.cross_over_possibility = cross_over_possibility
    self.cross_over_parent1_size = int(cross_over_parent1_ratio * self.patterns_length)
    for i in range(self.graph.node_count):
      self.base_pattern.append(i)
  
  def init_population(self):
    for i in range(self.population_count):
      random.shuffle(self.base_pattern)
      new_pattern = self.base_pattern.copy()
      new_individual = individual(new_pattern)
      self.fitness_function(new_individual) 
      self.population.append(new_individual)
    return self.population
  
  def compute_parents_and_random_subpopulation(self):
    sorted_pop = sorted(self.population, key=lambda x: x.fitness, reverse=True)
    parents = sorted_pop[:self.parents_count]
    random_population = [ind for ind in self.population if ind not in parents]
    random.shuffle(random_population)
    random_population = random_population[:self.recombination_low_fitness_count]
    return parents, random_population

  
  def print_generation(self, index, run_index):
    print("Run number #" + str(run_index) + ", Generation #" + str(index))
    for individual in self.population:
      print("Pattern: " + str(individual.pattern) + ", Fitness: " + str(individual.fitness))
    print("***********************************")
  
  def flip_coin(self, p):
    prob = random.random()
    if (prob <= p):
       return True
    return False 
    
  def fitness_function(self, individual):
    path_cost = 0
    for i in range(self.graph.node_count - 1):
      path_cost += self.graph.nodes[individual.pattern[i]][individual.pattern[i + 1]]
    individual.fitness = -path_cost
  
  def order1_crossover(self, parent1, parent2):
    child_pattern = []
    for i in range(self.patterns_length):
      child_pattern.append(-1)
    parent1_indices = []
    parent1_start_index = random.randrange(0, self.patterns_length)
    for i in range(self.cross_over_parent1_size):
      index = (parent1_start_index + i) % self.patterns_length
      parent1_indices.append(index)
    #print("parent1_indices" + str(parent1_indices))
    for i in parent1_indices:
        child_pattern[i] = parent1.pattern[i]
    parent2_pattern = parent2.pattern.copy()
    #print(parent2_pattern)
    #print("child_pattern" + str(child_pattern))
    for p in child_pattern:
      if (p != -1):
        parent2_pattern.remove(p)
    parent2_pattern.reverse()
    # print(parent2_pattern)
    for i in range(self.patterns_length):
      if (child_pattern[i] == -1):
        new_p = parent2_pattern.pop()
        # print("new_p " + str(new_p))
        child_pattern[i] = new_p
    child = individual(pattern=child_pattern)
    self.fitness_function(child)
    #print("child baby pattern: " + str(child.pattern))
    #print("child fitness: " + str(child.fitness))
    return child

  def crossover(self, parents, random_pop):
    do_crossover = self.flip_coin(self.cross_over_possibility)
    if not do_crossover:
      #print("No crossover in this step")
      return self.population
    new_population = []
    new_population.extend(random_pop)
    remaining_pop_count = self.population_count - len(random_pop)
    reamining_pop_candidates = []
    for i in range(2 * remaining_pop_count):
      parent1_index, parent2_index = random.sample((0, self.parents_count - 1), 2)
      parent1, parent2 = parents[parent1_index], parents[parent2_index]
      child = self.order1_crossover(parent1, parent2)
      reamining_pop_candidates.append(child)
    sorted_candidates = sorted(reamining_pop_candidates, key=lambda x: x.fitness, reverse=True)
    chosen_candidates = sorted_candidates[:remaining_pop_count]
    new_population.extend(chosen_candidates)
    self.population = new_population
    return new_population

  def compute_average_fitness(self):
    sum = 0.0
    for p in self.population:
      f = p.fitness
      sum += f
    avg = sum / self.population_count
    return avg
  
  def get_best_pattern_and_fitness(self):
    max_f = -inf
    best_pattern = None
    for p in self.population:
      if p.fitness > max_f:
        max_f = p.fitness
        best_pattern = p.pattern
    return best_pattern, max_f
  
  def mutation(self):
    for ind in self.population:
      do_mutation = self.flip_coin(self.mutation_possibility)
      if not do_mutation:
        continue
      mutated_bit_index = random.randrange(0, self.patterns_length)
      swapping_mutated_bit_index = random.choice([e for e in range(0, self.patterns_length) if e != mutated_bit_index])
      ind.pattern[mutated_bit_index], ind.pattern[swapping_mutated_bit_index] = ind.pattern[swapping_mutated_bit_index], ind.pattern[mutated_bit_index] 
      self.fitness_function(ind)
    return self.population 
  
  def run(self, index = 0):
    pop = self.init_population()
    best_fitness = -inf
    best_equal_in_a_row_count = 0
    generation_index = 0
    finish_criteria_sucessive_equal_count = 0.1 * self.max_generation_count
    finish_algorithm = False

    while not finish_algorithm:
      #if (generation_index % int(max_generation_count / 50) == 0):
        #self.print_generation(generation_index)
      current_best_pattern, current_best_fitness = self.get_best_pattern_and_fitness()
      parents, random_pop = self.compute_parents_and_random_subpopulation()
      self.crossover(parents, random_pop)
      self.mutation()
      new_best_pattern, new_best_fitness = self.get_best_pattern_and_fitness()
      if (current_best_pattern == new_best_pattern):
        best_equal_in_a_row_count += 1
      else:
        best_equal_in_a_row_count = 0
      if (best_equal_in_a_row_count == finish_criteria_sucessive_equal_count):
        finish_algorithm = True
      generation_index += 1
      if generation_index == self.max_generation_count:
        finish_algorithm = True
    #self.print_generation(generation_index, index)
    
    final_pattern, dist = self.get_best_pattern_and_fitness()
    dist = -dist
    print(final_pattern)
    print(dist)
    #print("Algorithm Terminated for run number #" + str(index) + ", here is the resulting path:\n\t" + "Path: "+ str([(x + 1) for x in final_pattern]) + "\n\tDistance: " + str(dist) + "\n")
    return (final_pattern, dist)
  
  def run_multiple_times(self, count):
    bast_pattern = None
    best_dist = inf
    for i in range(count):
      pattern, dist = self.run(i)
      if dist < best_dist:
        best_dist = dist
        best_pattern = pattern
    return [(x + 1) for x in best_pattern], best_dist



In [6]:
# a smaller example
# input_graph = graph(5)
# input_graph.add_edge(1, 4, 12)
# input_graph.add_edge(1, 2, 2)
# input_graph.add_edge(1, 5, 5)
# input_graph.add_edge(2, 3, 4)
# input_graph.add_edge(2, 4, 8)
# input_graph.add_edge(3, 4, 3)
# input_graph.add_edge(3, 5, 3)
# input_graph.add_edge(4, 5, 10)
# input_graph.print_graph()

# a bigger example
input_graph = graph(7)
input_graph.add_edge(1, 2, 12)
input_graph.add_edge(1, 3, 10)
input_graph.add_edge(1, 7, 12)
input_graph.add_edge(2, 4, 12)
input_graph.add_edge(2, 3, 8)
input_graph.add_edge(3, 4, 11)
input_graph.add_edge(3, 5, 3)
input_graph.add_edge(3, 7, 9)
input_graph.add_edge(5, 7, 7)
input_graph.add_edge(4, 5, 11)
input_graph.add_edge(6, 5, 6)
input_graph.add_edge(6, 4, 10)
input_graph.add_edge(6, 7, 9)
input_graph.print_graph()

# setting the ratios
population_count = 100
parents_ratio = 0.4
cross_over_parent1_ratio = 0.5
recombination_low_fitness_ratio = 0.5
cross_over_possibility = 0.75
mutation_possibility = 0.1
max_generation_count = 10000
run_count = 10

# run
genetic_algorithm = GA(input_graph, population_count, parents_ratio, cross_over_parent1_ratio, recombination_low_fitness_ratio, cross_over_possibility, mutation_possibility, max_generation_count)
path, distance = genetic_algorithm.run_multiple_times(run_count)

# print result
print("Best Solution:\n\t" + "Path: " + str(path) + "\n\t" + "Distane: " + str(distance))

Graph:

1: [inf, 12, 10, inf, inf, inf, 12]
2: [12, inf, 8, 12, inf, inf, inf]
3: [10, 8, inf, 11, 3, inf, 9]
4: [inf, 12, 11, inf, 11, 10, inf]
5: [inf, inf, 3, 11, inf, 6, 7]
6: [inf, inf, inf, 10, 6, inf, 9]
7: [12, inf, 9, inf, 7, 9, inf]


[0, 1, 2, 4, 6, 5, 3]
49
[3, 5, 6, 4, 2, 1, 0]
49
[0, 1, 2, 4, 6, 5, 3]
49
[6, 4, 5, 3, 1, 2, 0]
53
[3, 5, 6, 4, 2, 1, 0]
49
[3, 5, 6, 4, 2, 1, 0]
49
[3, 5, 6, 4, 2, 1, 0]
49
[0, 1, 2, 4, 6, 5, 3]
49
[3, 5, 4, 2, 1, 0, 6]
51
[0, 2, 1, 3, 5, 4, 6]
53
Best Solution:
	Path: [1, 2, 3, 5, 7, 6, 4]
	Distane: 49


# Equation solver (root finder) by GA



In [9]:
# GA and Individual structure

import math

class infividual:
  def __init__(self, pattern, fitness = -math.inf):
    self.pattern = pattern
    self.fitness = fitness
  
class GA:
  def __init__(self, population_count, crossover_probability, parents_ratio, recombination_low_fitness_ratio, mutation_step_ratio, mutation_probability, max_generation_count, max_worse_in_a_row_ratio):
    self.population_count = population_count
    self.parents_count = int(parents_ratio * self.population_count)
    self.crossover_probability = crossover_probability
    self.recombination_low_fitness_count = int(recombination_low_fitness_ratio * self.population_count)
    self.mutation_step_ratio = mutation_step_ratio
    self.mutation_probability = mutation_probability
    self.max_generation_count = max_generation_count
    self.max_worse_in_a_row_count = int(max_worse_in_a_row_ratio * self.max_generation_count)
    self.population = []
  
  def init_population(self):
    for i in range(int(self.population_count / 2)):
      pattern = i 
      ind1 = individual(pattern)
      ind2 = individual(-pattern)
      self.population.append(ind1)
      self.population.append(ind2)
      self.fitness_function(ind1)
      self.fitness_function(ind2)
    # if population count is odd, add one more
    if (self.population_count % 2 == 1):
      last_ind = individual(0)
      self.fitness_function(last_ind)
      self.population.append(last_ind)
    return self.population
  
  def print_generation(self, index, run_index):
    print("Generation #" + str(index) + " for run #" + str(run_index) + ":")
    for p in self.population:
      print("Pattern: " + str(p.pattern) + ", Fitness: " + str(p.fitness))
    print("********************************************************")
  
  # Equation: 9x^5 - 194.7x^4 + 1680.1x^3 - 7227.94x^2 + 15501.2x - 13257.2 = 0
  def F(self, x):
    return ((9 * (x**5)) + (-194.7 * (x**4)) + (1680.1 * (x**3)) + (-7227.94 * (x**2)) + (15501.2 * x) + (-13257.2))

  def fitness_function(self, individual):
    x = individual.pattern
    f = self.F(x)
    individual.fitness = -abs(f)
    return -f
  
  def crossover_avg(self, parent1, parent2):
    avg = (parent1.pattern + parent2.pattern) / 2
    child = individual(avg)
    self.fitness_function(child)
    return child
  
  def flip_coin(self, p):
    prob = random.random()
    if (prob <= p):
       return True
    return False

  def compute_parents_and_random_pop(self):
    parents = []
    sorted_population = sorted(self.population, key=lambda x: x.fitness, reverse=True)
    parents = sorted_population[:self.parents_count]
    random_population = [ind for ind in self.population if ind not in parents]
    random.shuffle(random_population)
    random_population = random_population[:self.recombination_low_fitness_count]
    return parents, random_population
    
  def crossover(self, parents, random_population):
    do_crossover = self.flip_coin(self.crossover_probability)
    if not do_crossover:
      #print("No crossover in this generation")
      return
    new_population = []
    new_population.extend(random_population)
    remaining_pop_count = self.population_count - len(random_population)
    reamining_pop_candidates = []
    for i in range(2 * remaining_pop_count):
      parent1_index, parent2_index = random.sample((0, self.parents_count - 1), 2)
      parent1, parent2 = parents[parent1_index], parents[parent2_index]
      child = self.crossover_avg(parent1, parent2)
      reamining_pop_candidates.append(child)
    sorted_candidates = sorted(reamining_pop_candidates, key=lambda x: x.fitness, reverse=True)
    chosen_candidates = sorted_candidates[:remaining_pop_count]
    new_population.extend(chosen_candidates)
    self.population = new_population
    return new_population
    
  def mutation(self):
    for ind in self.population:
      do_mutation = self.flip_coin(self.mutation_probability)
      if not do_mutation:
        continue
      mutation_step = ind.pattern * self.mutation_step_ratio
      increase_pattern = self.flip_coin(0.5)
      if not increase_pattern:
        mutation_step = -mutation_step
      ind.pattern += mutation_step
      self.fitness_function(ind)
  
  def compute_best_pattern_and_fitness(self):
    best_pattern = None
    best_fitness = -math.inf
    for ind in self.population:
      if ind.fitness > best_fitness:
        best_fitness = ind.fitness
        best_pattern = ind.pattern
    return (best_pattern, best_fitness)


  def run(self, run_index):
    self.init_population()
    finish_algorithm = False
    generation_index = 0
    worse_in_a_row_count = 0
    while (not finish_algorithm):
      current_best_pattern, current_best_fitness = self.compute_best_pattern_and_fitness()
      parents, random_pop = self.compute_parents_and_random_pop()
      self.crossover(parents, random_pop)
      self.mutation()
      #self.print_generation(generation_index, run_index)
      generation_index += 1
      if (generation_index == self.max_generation_count):
        finish_algorithm = True
      new_best_pattern, new_best_fitness = self.compute_best_pattern_and_fitness()
      if current_best_fitness > new_best_fitness:
       worse_in_a_row_count += 1
      if worse_in_a_row_count == self.max_worse_in_a_row_count:
        finish_algorithm = True
    #self.print_generation(generation_index, run_index)
    best_pattern, best_fitness = self.compute_best_pattern_and_fitness()
    return best_pattern, best_fitness
  
  def multiple_run(self, run_count):
    final_fitness = -math.inf
    final_pattern = None
    for i in range(run_count):
      best_pattern, best_fitness = self.run(i)
      if i % int(run_count / 10) == 0:
        print("Result from run #" + str(i) + ":\n\t" + "X = " + str(best_pattern), ", Fitness = " + str(best_fitness) + " (F = " + str(self.F(best_pattern)) + ")\n")
      if (best_fitness > final_fitness):
        final_fitness = best_fitness
        final_pattern = best_pattern
    print("Algorithm Terminated successfully:\n\tX = " + str(final_pattern) + ", F = " + str(self.F(best_pattern)) + "\n")
    return (final_pattern, final_fitness)

In [10]:
# setting variables
population_count = 100
crossover_probability = 0.75
parents_ratio = 0.2
recombination_low_fitness_ratio = 0.5
mutation_step_ratio = 0.1
mutation_probability = 0.2
max_generation_count = 100
max_worse_in_a_row_ratio = 0.2
run_count = 100

# run
genetic_algorithm = GA(population_count, crossover_probability, parents_ratio, recombination_low_fitness_ratio, mutation_step_ratio, mutation_probability, max_generation_count, max_worse_in_a_row_ratio)
(final_pattern, final_fitness) = genetic_algorithm.multiple_run(run_count)

Result from run #0:
	X = -0.005582540268520494 , Fitness = -13343.961622686222 (F = -13343.961622686222)

Result from run #10:
	X = 4.760940378230517 , Fitness = -0.06853007286918 (F = -0.06853007286918)

Result from run #20:
	X = 4.760940378230517 , Fitness = -0.06853007286918 (F = -0.06853007286918)

Result from run #30:
	X = 4.713449997957667 , Fitness = -0.07067831367385224 (F = -0.07067831367385224)

Result from run #40:
	X = 4.795468396100871 , Fitness = -0.06213715125704766 (F = -0.06213715125704766)

Result from run #50:
	X = 4.795468396100871 , Fitness = -0.06213715125704766 (F = -0.06213715125704766)

Result from run #60:
	X = 4.747633598849764 , Fitness = -0.06965176432640874 (F = -0.06965176432640874)

Result from run #70:
	X = 4.769847715427758 , Fitness = -0.06742727518212632 (F = -0.06742727518212632)

Result from run #80:
	X = 4.74599847685062 , Fitness = -0.06975306988533703 (F = -0.06975306988533703)

Result from run #90:
	X = 4.698657142044035 , Fitness = -0.07074417